In [20]:
import unicodedata
import sys, pathlib

# 1. Palindrome robuste

1. First draft

In [21]:
def is_palindrome_0(s: str) -> bool:
    if s is None:
        return False
    n = len(s)
    for i in range(n // 2):
        if s[i] != s[n - 1 - i]:
            return False
    return True


2. Second draft

In [22]:
def is_palindrome_1(s: str) -> bool:

    # normalise unicode, retire diacritiques et passe en minuscule
    s_norm = unicodedata.normalize("NFD", s)
    s_norm = "".join(ch for ch in s_norm if unicodedata.category(ch) != "Mn")
    t = "".join(ch.lower() for ch in s_norm if ch.isalnum())

    palindrome = True

    i, j = 0, len(t) - 1
    while i < j:
        if t[i] != t[j]:
            palindrome = False
        i += 1
        j -= 1
    
    return palindrome 



3. Third draft

In [23]:
def is_palindrome_2(s: str) -> bool:
    """

    Détermine si une chaîne est un palindrome en ignorant la casse et les caractères non alphanumériques (accents normalisés).

    Complexité: O(n) en temps, O(n) en mémoire (pour la version filtrée).

    """
    if s is None:
        return False

    # Normalisation Unicode (enlève les diacritiques) + minuscule
    s_norm = unicodedata.normalize("NFD", s)
    s_norm = "".join(ch for ch in s_norm if unicodedata.category(ch) != "Mn")
    s_norm = s_norm.lower()

    # Filtrage: on ne garde que les alphanumériques
    filtered = [ch for ch in s_norm if ch.isalnum()]

    # Test palindrome par comparaison miroir
    return filtered == filtered[::-1]

In [24]:

assert is_palindrome_1("kayak") is True
assert is_palindrome_1("A man, a plan, a canal: Panama!") is True
assert is_palindrome_1("été") is True
assert is_palindrome_1("python") is False


In [25]:

is_palindrome_0("kayak")

True

Tests

In [27]:
import pytest

from katas.palindrome import is_palindrome_1

@pytest.mark.parametrize("s,expected", [
    ("kayak", True),
    ("", True),
    ("A man, a plan, a canal: Panama!", True),
    ("No 'x' in Nixon", True),
    ("été", True),                 # avec accents
    ("python", False),
    ("ab", False),
    (None, False),                 # si vous choisissez de renvoyer False pour None
])
def test_is_palindrome(s, expected):
    assert is_palindrome_1(s) is expected


ModuleNotFoundError: No module named 'katas'

#### Lexiques
Signe diacritique : signe graphique (point, accent…) qui empêche la confusion entre homographes*. Les accents des mots à, dû, où sont des signes diacritiques. 

# 2. Factorial

0! = 1 par convention,

n doit être un entier ≥ 0,

sinon, on doit lever une erreur appropriée.

#### 2.1 Factorial - first draft

In [13]:
def factorial_0(n: int)-> int:
    r = 1
    if n == 0:
        return 1
    else:
        for k in range(2,n+1):
            r *= k
        return r

In [16]:
for i in range(11):
    print(i,":",factorial_0(i))

0 : 1
1 : 1
2 : 2
3 : 6
4 : 24
5 : 120
6 : 720
7 : 5040
8 : 40320
9 : 362880
10 : 3628800


In [17]:
def factorial(n: int) -> int:
    """
    Calcule la factorielle d'un entier n (n!).
    
    Raises:
        TypeError: si n n'est pas un int
        ValueError: si n est négatif
    """
    if not isinstance(n, int):
        raise TypeError("n must be an int")
    if n < 0:
        raise ValueError("n must be >= 0")

    result = 1
    for i in range(2, n + 1):
        result *= i
    return result


In [ ]:
from src.factorial import factorial 

@pytest.mark.parametrize("n,expected", [
    (0, 1),
    (1, 1),
    (5, 120),
    (6, 720),
])
def test_factorial_values(n, expected):
    assert factorial(n) == expected

def test_factorial_negative():
    with pytest.raises(ValueError):
        factorial(-1)

def test_factorial_non_int():
    with pytest.raises(TypeError):
        factorial(3.5)  # type: ignore
    with pytest.raises(TypeError):
        factorial("10")  # type: ignore


ModuleNotFoundError: No module named 'src'

# 3. Fibonacci

But : renvoyer F(n) avec boucle.  

I/O : fib(n: int) -> int  

Contraintes : n>=0, gérer types ; O(n).  

Tests : 0→0, 1→1, 2→1, 7→13, erreurs pour -2, 1.5.  

Critères : aucun appel récursif.  

Extensions : version en temps O(log n) (exponentiation de matrices).

In [62]:
def Fibonacci(n:int) -> int:

    """
    Renvoie le n-ième nombre de Fibonacci (0-indexé).
    F(0)=0, F(1)=1, puis F(n)=F(n-1)+F(n-2) pour n>=2.

    Raises:
        TypeError: si n n'est pas un int.
        ValueError: si n < 0.
    """

    if not isinstance(n, int):
        raise TypeError("n must be an int")
    if n < 0:
        raise ValueError("n must be >= 0")
    
    A = [0,1]

    if n < 2:
        return A[n]
    
    else:
        for i in range(2, n + 1):
                A.append(A[i - 1] + A[i - 2])
        return A[len(A)-1]

    

In [63]:
for n in range(11):
    print(n,":",Fibonacci(n))


0 : 0
1 : 1
2 : 1
3 : 2
4 : 3
5 : 5
6 : 8
7 : 13
8 : 21
9 : 34
10 : 55


# 4. Anagram

In [77]:
import unicodedata
from collections import Counter

In [78]:
def _normalize(s: str) -> str:
    """
    Normalise une chaîne :
    - décomposition Unicode (NFD)
    - suppression des diacritiques (catégorie Mn)
    - passage en minuscules
    - conservation des seuls alphanumériques
    """
    s_nfd = unicodedata.normalize("NFD", s)
    no_diacritics = "".join(ch for ch in s_nfd if unicodedata.category(ch) != "Mn")
    return "".join(ch.lower() for ch in no_diacritics if ch.isalnum())

In [80]:
def is_anagram(a: str, b: str) -> bool:

    if a is None:
        return False
        
    a_norm = _normalize(a)
    b_norm = _normalize(b)
    
    if len(a_norm) != len(b_norm):
        return False

    return Counter(a_norm) == Counter(b_norm)

In [ ]:
import pytest
from src.anagram import is_anagram  # adapte l'import selon ton module

@pytest.mark.parametrize("a,b,expected", [
    ("listen", "silent", True),
    ("Listen", "Silent", True),                                  # casse ignorée
    ("Dormitory", "Dirty room!!", True),                         # espaces/ponctuation ignorés
    ("Été", "ête", True),                                        # diacritiques ignorés
    ("éèàç", "aeec", True),                                      # normalisation étendue
    ("A gentleman", "Elegant man", True),
    ("William Shakespeare", "I am a weakish speller", True),
    ("apple", "pale", False),
    ("ab", "a b c", False),
    ("", "", True),                                              # deux vides considérés anagrammes
])
def test_is_anagram(a, b, expected):
    assert is_anagram(a, b) is expected

def test_is_anagram_none():
    assert is_anagram(None, "abc") is False      # type: ignore[arg-type]
    assert is_anagram("abc", None) is False      # type: ignore[arg-type]
    assert is_anagram(None, None) is False       # type: ignore[arg-type]


In [85]:
is_anagram("listen", "silent")

True